In [1]:
import json
import requests
from requests import ConnectionError
from typing import Optional, Any
from logging import getLogger
import pandas as pd

from yahoo_finance_dash.config import get_settings

In [2]:
settings = get_settings()

logger = getLogger(__name__)

In [43]:
class YahooDataGrabber:
    def __init__(self, api_key: str) -> None:
        self.headers = {
            'accept': 'application/json',
            'X-API-KEY': api_key,
        }

    def _fetch_response_dict(self, api_args: str) -> Optional[dict[str, Any]]:
        try:
            response = json.loads(
                requests.get(
                    f'https://yfapi.net/v8/finance/{api_args}',
                    headers=self.headers,
                ).text
            )
            if response:
                # Catch API failures as they come through in JSON format too
                if "message" in response.keys():
                    logger.error(
                        f"Query failed to return any results.\n"
                        f"Query used: {api_args}\n"
                        f"Message: {response['message']}"
                    )
                    return None
                
                # Successful response
                return response

            else:
                # No response provided by API
                logger.error(f"Query failed to return any results.\nQuery used: {api_args}")
                return None

        except ConnectionError as e:
            logger.error(f"Failed to connect to Yahoo Finance API! {e}")
            return None
    
    def spark(self, asx_codes: list[str]) -> pd.DataFrame:
        api_args = "spark?interval=1d&range=1mo&symbols=AAPL"
        response = self._fetch_response_dict(api_args)
        
        if response:
            # Iterate through each code, then concatentate
            response_dfs = []
            for code in asx_codes:
                reponse_current = response[code]
                response_df = pd.DataFrame(
                    {
                        "timestamp": reponse_current['timestamp'],
                        "close": reponse_current['close'],
                    }
                )
                response_df.insert(loc=0, column='asx_code', value=code)
                response_dfs.append(response_df)
            return pd.concat(response_dfs)
        else:
            return None

In [44]:
ydg = YahooDataGrabber(settings.YAHOO_FINANCE_API_KEY)

In [45]:
ydg.spark(['AAPL'])

,asx_code,timestamp,close
0,AAPL,1646836200,162.95
1,AAPL,1646922600,158.52
2,AAPL,1647009000,154.73
3,AAPL,1647264600,150.62
4,AAPL,1647351000,155.09
5,AAPL,1647437400,159.59
6,AAPL,1647523800,160.62
7,AAPL,1647610200,163.98
8,AAPL,1647869400,165.38
9,AAPL,1647955800,168.82


In [29]:
my_headers = {
    'accept': 'application/json',
    'X-API-KEY': settings.YAHOO_FINANCE_API_KEY
}
response = json.loads(
    requests.get(
        'https://yfapi.net/v8/finance/spark?interval=1d&range=1mo&symbols=AAPL',
        headers=my_headers,
    ).text
)

In [20]:
response['AAPL'].keys()

dict_keys(['symbol', 'timestamp', 'end', 'start', 'dataGranularity', 'close', 'previousClose', 'chartPreviousClose'])

In [22]:
response['AAPL']['timestamp'][:3]

[1646836200, 1646922600, 1647009000]

In [28]:
print(response['AAPL']['close'])

[162.95, 158.52, 154.73, 150.62, 155.09, 159.59, 160.62, 163.98, 165.38, 168.82, 170.21, 174.07, 174.72, 175.6, 178.96, 177.77, 174.61, 174.31, 178.44, 175.06, 171.83, 172.14, 170.09]
